In [63]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

ModuleNotFoundError: No module named 'xgboost'

In [2]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,5 days 10 hours 40 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,"21 days, 6 hours and 27 minutes"
H2O_cluster_name:,H2O_from_python_per_christian_43xdwe
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.782 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

In [4]:
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')


In [5]:
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')



In [6]:
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')


In [7]:
x_train_merged_a = pd.concat([X_train_observed_a,X_train_estimated_a])
x_train_merged_b = pd.concat([X_train_observed_b,X_train_estimated_b])
x_train_merged_c = pd.concat([X_train_observed_c,X_train_estimated_c])

In [8]:
x_train_merged_a

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,date_calc
0,2019-06-02 22:00:00,7.7,1.230,1744.900024,0.0,0.0,1744.900024,0.0,280.299988,0.0,0.0,0.0,0.0,98.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.799988,0.0,0.0,994.200012,1000.299988,0.0,0.0,73.099998,1006.299988,NaN,0.0,0.0,-0.0,0.1,342.834015,-3.202,0.0,285.899994,100.000000,39640.101562,3.7,-3.6,-0.8,-0.0,NaT
1,2019-06-02 22:15:00,7.7,1.229,1734.000000,0.0,0.0,1734.000000,0.0,280.299988,0.0,0.0,0.0,0.0,99.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.500000,0.0,0.0,993.900024,999.900024,0.0,0.0,72.199997,1006.000000,NaN,0.0,0.0,-0.0,0.2,346.294006,-3.650,0.0,286.100006,100.000000,40123.898438,3.6,-3.6,-0.6,-0.0,NaT
2,2019-06-02 22:30:00,7.7,1.228,1723.500000,0.0,0.0,1723.500000,0.0,280.299988,0.0,0.0,0.0,0.0,99.199997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.099976,0.0,0.0,993.599976,999.599976,0.0,0.0,71.199997,1005.599976,NaN,0.0,0.0,-0.0,0.2,349.768005,-3.998,0.0,286.299988,100.000000,40628.300781,3.6,-3.6,-0.4,-0.0,NaT
3,2019-06-02 22:45:00,7.7,1.226,1713.400024,0.0,0.0,1713.400024,0.0,280.299988,0.0,0.0,0.0,0.0,99.400002,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.799988,0.0,0.0,993.299988,999.299988,0.0,0.0,70.199997,1005.299988,NaN,0.0,0.0,-0.0,0.2,353.251007,-4.247,0.0,286.600006,100.000000,41153.601562,3.5,-3.5,-0.2,-0.0,NaT
4,2019-06-02 23:00:00,7.7,1.225,1703.599976,0.0,0.0,1703.599976,0.0,280.299988,0.0,0.0,0.0,0.0,99.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.500000,0.0,0.0,993.000000,999.000000,0.0,0.0,69.199997,1005.000000,NaN,0.0,0.0,-0.0,0.2,356.742004,-4.393,0.0,286.799988,100.000000,41699.898438,3.5,-3.5,0.0,-0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17571,2023-04-30 22:45:00,4.5,1.281,1173.900024,0.0,0.0,536.500000,0.0,272.299988,0.0,0.0,0.0,0.0,99.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.299988,0.0,0.0,1001.099976,1007.400024,0.0,0.0,79.599998,1013.700012,NaN,0.0,0.0,-0.0,0.1,352.920013,-11.731,0.1,274.200012,99.900002,11629.299805,3.9,2.9,2.5,-0.0,2023-04-29 07:00:05
17572,2023-04-30 23:00:00,4.5,1.281,1054.199951,0.0,0.0,542.400024,0.0,272.200012,0.0,0.0,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.299988,0.0,0.0,1001.099976,1007.299988,0.0,0.0,80.000000,1013.599976,NaN,0.0,0.0,-0.0,0.1,356.634003,-11.884,0.1,274.200012,100.000000,9923.200195,3.7,2.8,2.4,-0.0,2023-04-29 07:00:05
17573,2023-04-30 23:15:00,4.5,1.281,1435.800049,0.0,0.0,531.799988,0.0,272.299988,0.0,0.0,0.0,0.0,99.400002,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.099976,0.0,0.0,1000.900024,1007.200012,0.0,0.0,79.900002,1013.500000,NaN,0.0,0.0,-0.0,0.1,0.352000,-11.928,0.1,274.200012,99.900002,11230.799805,3.7,2.7,2.5,-0.0,2023-04-29 07:00:05
17574,2023-04-30 23:30:00,4.5,1.281,1817.400024,0.0,0.0,521.200012,0.0,272.299988,0.0,0.0,0.0,0.0,98.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.000000,0.0,0.0,1000.799988,1007.000000,0.0,0.0,79.800003,1013.299988,NaN,0.0,0.0,-0.0,0.0,4.070000,-11.864,0.1,274.200012,99.599998,12526.099609,3.8,2.7,2.6,-0.0,2023-04-29 07:00:05


In [9]:
# Calculating from 15-minute intervals to hourly intervals based on different aggregation methods
def resample_to_hourly(df, aggregation_methods):
    df_hourly = df.resample('H', on = 'date_forecast').agg(aggregation_methods)
    df_hourly.reset_index(inplace=True)
    return df_hourly

# Aggregation methods based on features' names
aggregation_methods = {
    'date_calc' : 'max',
    'absolute_humidity_2m:gm3': 'mean',
    'air_density_2m:kgm3': 'mean',
    'ceiling_height_agl:m': 'mean',
    'clear_sky_energy_1h:J': 'sum',
    'clear_sky_rad:W': 'mean',
    'cloud_base_agl:m': 'mean',
    'dew_or_rime:idx': 'max',
    'dew_point_2m:K': 'mean',
    'diffuse_rad:W': 'mean',
    'diffuse_rad_1h:J': 'sum',
    'direct_rad:W': 'mean',
    'direct_rad_1h:J': 'sum',
    'effective_cloud_cover:p': 'mean',
    'elevation:m': 'mean',
    'fresh_snow_12h:cm': 'sum',
    'fresh_snow_1h:cm': 'sum',
    'fresh_snow_24h:cm': 'sum',
    'fresh_snow_3h:cm': 'sum',
    'fresh_snow_6h:cm': 'sum',
    'is_day:idx': 'max',
    'is_in_shadow:idx': 'max',
    'msl_pressure:hPa': 'mean',
    'precip_5min:mm': 'sum',
    'precip_type_5min:idx': 'max',
    'pressure_100m:hPa': 'mean',
    'pressure_50m:hPa': 'mean',
    'prob_rime:p': 'mean',
    'rain_water:kgm2': 'sum',
    'relative_humidity_1000hPa:p': 'mean',
    'sfc_pressure:hPa': 'mean',
    'snow_density:kgm3': 'mean',
    'snow_depth:cm': 'mean',
    'snow_drift:idx': 'max',
    'snow_melt_10min:mm': 'sum',
    'snow_water:kgm2': 'sum',
    'sun_azimuth:d': 'mean',
    'sun_elevation:d': 'mean',
    'super_cooled_liquid_water:kgm2': 'mean',
    't_1000hPa:K': 'mean',
    'total_cloud_cover:p': 'mean',
    'visibility:m': 'mean',
    'wind_speed_10m:ms': 'mean',
    'wind_speed_u_10m:ms': 'mean',
    'wind_speed_v_10m:ms': 'mean',
    'wind_speed_w_1000hPa:ms': 'mean'
}


In [10]:
# Apply the function to each dataset
x_train_a_hourly = resample_to_hourly(x_train_merged_a, aggregation_methods)
x_train_b_hourly = resample_to_hourly(x_train_merged_b, aggregation_methods)
x_train_c_hourly = resample_to_hourly(x_train_merged_c, aggregation_methods)

x_test_a_hourly = resample_to_hourly(X_test_estimated_a, aggregation_methods)
x_test_b_hourly = resample_to_hourly(X_test_estimated_b, aggregation_methods)
x_test_c_hourly = resample_to_hourly(X_test_estimated_c, aggregation_methods)

In [11]:
# Remove all rows with date-time values that are not present in both x and y in order to synchronize x and its labels. 
def remove_non_synchronous_rows(x_train, y_train, x_date_column='date_forecast', y_date_column='time'):
    # Convert date columns to datetime format for easier comparison
    x_train[x_date_column] = pd.to_datetime(x_train[x_date_column])
    y_train[y_date_column] = pd.to_datetime(y_train[y_date_column])
    
    # Find common dates
    common_dates = x_train[x_date_column][x_train[x_date_column].isin(y_train[y_date_column])]
    
    # Filter both datasets based on common dates
    x_train_synced = x_train[x_train[x_date_column].isin(common_dates)]
    y_train_synced = y_train[y_train[y_date_column].isin(common_dates)]
    
    return x_train_synced, y_train_synced


In [12]:
x_train_a_hourly

,date_forecast,date_calc,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2019-06-02 22:00:00,NaT,7.700,1.22825,1728.949951,0.000000,0.000,1728.949951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.074997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.299988,0.0,0.0,993.750000,999.775024,0.0,0.0,71.674995,1005.799988,NaN,0.0,0.0,0.0,0.7,348.036743,-3.77425,0.000,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0
1,2019-06-02 23:00:00,NaT,7.700,1.22350,1689.824951,0.000000,0.000,1689.824951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.750000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.200012,0.0,0.0,992.674988,998.650024,0.0,0.1,68.000000,1004.650024,NaN,0.0,0.0,0.0,0.8,91.980751,-4.35725,0.000,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0
2,2019-06-03 00:00:00,NaT,7.875,1.21975,1563.224976,0.000000,0.000,1563.224976,0.0,280.649994,0.000,0.000000,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1004.525024,0.0,0.0,992.000000,997.974976,0.0,0.4,67.949997,1003.950012,NaN,0.0,0.0,0.0,1.6,14.934750,-3.30950,0.000,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0
3,2019-06-03 01:00:00,NaT,8.425,1.21800,1283.425049,834.599976,0.750,1283.425049,0.0,281.674988,0.300,2107.100098,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1004.025024,0.0,0.0,991.500000,997.449951,0.0,0.5,73.875000,1003.449951,NaN,0.0,0.0,0.0,2.2,28.630251,-0.82250,0.000,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0
4,2019-06-03 02:00:00,NaT,8.950,1.21800,1003.500000,129872.601562,23.100,1003.500000,0.0,282.500000,11.975,88275.796875,0.15,1131.900024,84.875000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1003.099976,0.0,0.0,990.550049,996.500000,0.0,0.4,79.925003,1002.500000,NaN,0.0,0.0,0.0,1.0,41.997501,3.05125,0.000,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34269,2023-04-30 19:00:00,2023-04-29 07:00:05,4.550,1.27650,1674.199951,337859.781250,4.225,542.700012,0.0,272.424988,2.825,225724.203125,0.10,52922.597656,96.699997,6.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1014.750000,0.0,0.0,1001.574951,1007.799988,0.0,0.0,74.625000,1014.049988,NaN,0.0,0.0,0.0,0.0,304.939240,-0.18050,0.000,275.024994,96.699997,23417.074219,5.175,4.800,1.925,0.0
34270,2023-04-30 20:00:00,2023-04-29 07:00:05,4.500,1.27975,1762.400024,9083.500000,0.000,546.400024,0.0,272.299988,0.000,20271.500000,0.00,705.400024,94.224998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.700012,0.0,0.0,1001.549988,1007.799988,0.0,0.0,76.875000,1014.049988,NaN,0.0,0.0,0.0,0.0,318.625763,-5.17600,0.000,274.650024,94.525002,21084.050781,4.650,4.025,2.300,0.0
34271,2023-04-30 21:00:00,2023-04-29 07:00:05,4.500,1.28100,1696.650024,0.000000,0.000,548.349976,0.0,272.299988,0.000,0.000000,0.00,0.000000,94.324997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.549988,0.0,0.0,1001.400024,1007.674988,0.0,0.0,77.775002,1013.925049,NaN,0.0,0.0,0.0,0.0,332.785736,-8.95075,0.000,274.524994,95.675003,20792.500000,4.450,3.575,2.600,0.0
34272,2023-04-30 22:00:00,2023-04-29 07:00:05,4.500,1.28100,1353.400024,0.000000,0.000,527.775024,0.0,272.299988,0.000,0.000000,0.00,0.000000,97.775002,6.0,0.0,0.0,0.

In [13]:
# Remove the rows with date and time that only shows up in one of the sets
x_train_a_hourly, train_a = remove_non_synchronous_rows(x_train_a_hourly, train_a)
x_train_b_hourly, train_b = remove_non_synchronous_rows(x_train_b_hourly, train_b)
x_train_c_hourly, train_c = remove_non_synchronous_rows(x_train_c_hourly, train_c)

In [14]:
    
test = pd.read_csv('test.csv')
pred_time_stamps = test['time'].unique()
x_test_a_hourly = x_test_a_hourly[x_test_a_hourly['date_forecast'].isin(pred_time_stamps)]
x_test_b_hourly = x_test_b_hourly[x_test_b_hourly['date_forecast'].isin(pred_time_stamps)]
x_test_c_hourly = x_test_c_hourly[x_test_c_hourly['date_forecast'].isin(pred_time_stamps)]


In [15]:
x_test_a_hourly

,date_forecast,date_calc,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-05-01 00:00:00,2023-04-30 07:00:04,4.325,1.28675,912.700012,0.000000e+00,0.000000,1061.550049,0.0,271.650024,0.000000,0.000000,0.000000,0.000000,74.949997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.674988,0.0,0.0,1000.549988,1006.799988,0.0,0.0,80.275002,1013.099976,NaN,0.0,0.0,0.0,0.0,16.026501,-10.54100,0.000,273.799988,74.949997,29907.500000,3.950,2.100,3.350,0.0
1,2023-05-01 01:00:00,2023-04-30 07:00:04,4.275,1.28600,1482.099976,0.000000e+00,0.000000,1075.100098,0.0,271.450012,0.000000,0.000000,0.000000,0.000000,77.474998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.150024,0.0,0.0,1000.049988,1006.299988,0.0,0.0,79.824997,1012.599976,NaN,0.0,0.0,0.0,0.0,30.497250,-7.89450,0.000,273.799988,77.474998,29519.074219,3.825,1.925,3.300,0.0
2,2023-05-01 02:00:00,2023-04-30 07:00:04,4.150,1.28375,1791.300049,0.000000e+00,0.000000,1200.400024,0.0,271.049988,0.000000,0.000000,0.000000,0.000000,88.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1012.674988,0.0,0.0,999.500000,1005.799988,0.0,0.0,78.000000,1012.049988,NaN,0.0,0.0,0.0,0.0,44.517250,-3.81550,0.000,273.849976,88.099998,31009.125000,3.650,1.750,3.200,0.0
3,2023-05-01 03:00:00,2023-04-30 07:00:04,4.025,1.28200,2312.875000,4.049770e+04,11.675000,1179.849976,0.0,270.649994,9.375000,67380.906250,2.100000,15061.400391,68.599998,6.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1012.174988,0.0,0.0,998.974976,1005.224976,0.0,0.0,75.625000,1011.525024,NaN,0.0,0.0,0.0,0.0,58.083000,1.41250,0.000,273.899994,68.599998,34552.500000,3.500,1.450,3.150,0.0
4,2023-05-01 04:00:00,2023-04-30 07:00:04,3.900,1.28100,2198.299805,5.669944e+05,76.875000,920.049988,0.0,270.375000,47.400002,408838.812500,25.450001,198284.796875,66.300003,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1011.724976,0.0,0.0,998.549988,1004.750000,0.0,0.0,74.224998,1011.049988,NaN,0.0,0.0,0.0,0.0,71.341003,7.46850,0.000,273.924988,66.300003,35483.875000,3.325,1.300,3.050,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,2023-07-03 19:00:00,2023-07-02 07:00:31,8.350,1.19725,3638.899902,1.908373e+06,85.099998,2013.750000,0.0,281.575012,33.625000,675098.187500,13.425000,203853.000000,85.574997,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,992.099976,0.0,0.0,979.599976,985.424988,0.0,0.0,69.324997,991.325012,NaN,0.0,0.0,0.0,0.0,306.687012,8.15300,0.100,287.024994,86.150002,44056.375000,2.450,2.075,-1.350,0.0
1532,2023-07-03 20:00:00,2023-07-02 07:00:31,8.525,1.20050,3552.100098,7.373427e+05,24.799999,1610.900024,0.0,281.850006,14.325000,345284.500000,3.550000,122263.500000,74.599998,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,992.549988,0.0,0.0,980.025024,985.900024,0.0,0.0,72.900002,991.799988,NaN,0.0,0.0,0.0,0.0,319.704010,3.26800,0.000,286.600006,75.324997,44017.175781,2.450,2.100,-1.250,0.0
1533,2023-07-03 21:00:00,2023-07-02 07:00:31,8.825,1.20450,2315.000000,1.497173e+05,1.275000,1622.800049,0.0,282.325012,1.300000,112676.601562,0.000000,25639.599609,76.125000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,992.900024,0.0,0.0,980.349976,986.250000,0.0,0.0,78.050003,992.150024,NaN,0.0,0.0,0.0,0.0,333.040009,-0.44325,0.000,286.100006,76.849998,43302.050781,2

In [16]:
#Create time-based features for hour,day, month
def add_time_feat(X):
    X_frame = X.copy().set_index('date_forecast')
    X_frame['year'] = X_frame.index.year
    X_frame['month'] = X_frame.index.month
    X_frame['day'] = X_frame.index.day
    X_frame['hour'] = X_frame.index.hour
    return X_frame

In [78]:

X_train_a = add_time_feat(x_train_a_hourly).drop(columns = ['date_calc']).reset_index(drop = True)
X_test_a = add_time_feat(x_test_a_hourly).drop(columns = ['date_calc']).reset_index(drop = True)

X_train_b = add_time_feat(x_train_b_hourly).drop(columns = ['date_calc']).reset_index(drop = True)
X_test_b = add_time_feat(x_test_b_hourly).drop(columns = ['date_calc']).reset_index(drop = True)

X_train_c = add_time_feat(x_train_c_hourly).drop(columns = ['date_calc']).reset_index(drop = True)
X_test_c = add_time_feat(x_test_c_hourly).drop(columns = ['date_calc']).reset_index(drop = True)

In [60]:
from h2o.automl import H2OAutoML

def build_model(X_train,y_train,location):
    merged_data = X_train.copy()
    merged_data['pv_measurement'] = y_train['pv_measurement'].values
    
    y = 'pv_measurement'
    x = list(X_train.columns)
    train = h2o.H2OFrame(merged_data)
    
    aml = H2OAutoML(
        max_models = 10,
        max_runtime_secs = 60,
        #exclude_algos =['DeepLearning'],
        seed = 1,
        # stopping_metric ='logloss',
        sort_metric ='mae',
        balance_classes = False,
        project_name ='Project1'
    )

    aml.train(x=x, y=y, training_frame=train)
    
    lb = aml.leaderboard
    leader = aml.leader
    print(lb.head(rows=lb.nrows))
    
    #h2o.save_model(leader, path=f'Saved_models/{location.upper()}', force = True)

    return lb,leader

In [30]:
lb_a,model_a = build_model(X_train_a,train_a,'A')
lb_b,model_b = build_model(X_train_b,train_b,'B')
lb_c,model_c = build_model(X_train_c,train_c,'C')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
21:38:57.664: New models will be added to existing leaderboard Project1@@pv_measurement (leaderboard frame=null) with already 137 models.
21:38:57.665: AutoML: XGBoost is not available; skipping it.
21:38:57.666: _train param, Dropping bad and constant columns: [elevation:m]

█████████████████████████████████████████████████████████
21:40:47.611: XRT_1_AutoML_21_20231024_213857 [DRF XRT (Extremely Randomized Trees)] failed: java.lang.AssertionError

██████| (done) 100%
model_id                                            mae      rmse        mse       rmsle    mean_residual_deviance
GBM_grid_6_AutoML_9_20231022_131607_model_5     24.0293   58.2447    3392.45  nan                          3392.45
GBM_grid_3_AutoML_6_20231022_125205_model_5     24.0293   58.2447    3392.45  nan                          3392.45
GBM_grid_4_AutoML_7_20231022_125409_model_5     24.2892   58.1782  

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
21:38:57.664: New models will be added to existing leaderboard Project1@@pv_measurement (leaderboard frame=null) with already 137 models.
21:38:57.665: AutoML: XGBoost is not available; skipping it.
21:38:57.666: _train param, Dropping bad and constant columns: [elevation:m]
21:40:47.611: XRT_1_AutoML_21_20231024_213857 [DRF XRT (Extremely Randomized Trees)] failed: java.lang.AssertionError
21:41:00.956: New models will be added to existing leaderboard Project1@@pv_measurement (leaderboard frame=null) with already 143 models.
21:41:00.956: AutoML: XGBoost is not available; skipping it.
21:41:00.965: _train param, Dropping bad and constant columns: [elevation:m]

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                            mae      rmse        mse       rmsle    mean_residual_deviance
GBM_grid_6_AutoML_9_202

In [36]:
preds_a = model_a.predict(h2o.H2OFrame(X_test_a))
preds_b = model_b.predict(h2o.H2OFrame(X_test_b))
preds_c = model_c.predict(h2o.H2OFrame(X_test_c))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [74]:
X

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,year,month,day,hour
date_forecast,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-06-02 22:00:00,7.700,1.22825,1728.949951,0.000000,0.000,1728.949951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.074997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.299988,0.0,0.0,993.750000,999.775024,0.0,0.0,71.674995,1005.799988,NaN,0.0,0.0,0.0,0.7,348.036743,-3.77425,0.000,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0,2019,6,2,22
2019-06-02 23:00:00,7.700,1.22350,1689.824951,0.000000,0.000,1689.824951,0.0,280.299988,0.000,0.000000,0.00,0.000000,99.750000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.200012,0.0,0.0,992.674988,998.650024,0.0,0.1,68.000000,1004.650024,NaN,0.0,0.0,0.0,0.8,91.980751,-4.35725,0.000,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0,2019,6,2,23
2019-06-03 00:00:00,7.875,1.21975,1563.224976,0.000000,0.000,1563.224976,0.0,280.649994,0.000,0.000000,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1004.525024,0.0,0.0,992.000000,997.974976,0.0,0.4,67.949997,1003.950012,NaN,0.0,0.0,0.0,1.6,14.934750,-3.30950,0.000,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0,2019,6,3,0
2019-06-03 01:00:00,8.425,1.21800,1283.425049,834.599976,0.750,1283.425049,0.0,281.674988,0.300,2107.100098,0.00,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1004.025024,0.0,0.0,991.500000,997.449951,0.0,0.5,73.875000,1003.449951,NaN,0.0,0.0,0.0,2.2,28.630251,-0.82250,0.000,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0,2019,6,3,1
2019-06-03 02:00:00,8.950,1.21800,1003.500000,129872.601562,23.100,1003.500000,0.0,282.500000,11.975,88275.796875,0.15,1131.900024,84.875000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1003.099976,0.0,0.0,990.550049,996.500000,0.0,0.4,79.925003,1002.500000,NaN,0.0,0.0,0.0,1.0,41.997501,3.05125,0.000,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0,2019,6,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.550,1.27650,1674.199951,337859.781250,4.225,542.700012,0.0,272.424988,2.825,225724.203125,0.10,52922.597656,96.699997,6.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1014.750000,0.0,0.0,1001.574951,1007.799988,0.0,0.0,74.625000,1014.049988,NaN,0.0,0.0,0.0,0.0,304.939240,-0.18050,0.000,275.024994,96.699997,23417.074219,5.175,4.800,1.925,0.0,2023,4,30,19
2023-04-30 20:00:00,4.500,1.27975,1762.400024,9083.500000,0.000,546.400024,0.0,272.299988,0.000,20271.500000,0.00,705.400024,94.224998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.700012,0.0,0.0,1001.549988,1007.799988,0.0,0.0,76.875000,1014.049988,NaN,0.0,0.0,0.0,0.0,318.625763,-5.17600,0.000,274.650024,94.525002,21084.050781,4.650,4.025,2.300,0.0,2023,4,30,20
2023-04-30 21:00:00,4.500,1.28100,1696.650024,0.000000,0.000,548.349976,0.0,272.299988,0.000,0.000000,0.00,0.000000,94.324997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.549988,0.0,0.0,1001.400024,1007.674988,0.0,0.0,77.775002,1013.925049,NaN,0.0,0.0,0.0,0.0,332.785736,-8.95075,0.000,274.524994,95.675003,20792.500000,4.450,3.575,2.600,0.0,2023,4,30,21


In [88]:
import xgboost as xgb

def XGBoost_model(x_train,y_train, x_test):
    split_index = int(0.875 * len(x_train))

    # Split the data into training and validation sets
    x_train_df = x_train[:split_index]
    y_train_df = y_train[:split_index]
    x_valid_df = x_train[split_index:]
    y_valid_df = y_train[split_index:]
    
    xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'booster': 'gbtree',
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'alpha': 0.1,
    'lambda': 1,

    }
    
    dtrain = xgb.DMatrix(x_train_df,label = y_train_df['pv_measurement'])
    dval = xgb.DMatrix(x_valid_df,label = y_valid_df['pv_measurement'])
    test = xgb.DMatrix(x_test)

    #  Train the model
    num_boost_round = 1000  # Maximum number of boosting rounds
    early_stopping_rounds = 50  # Stop if validation score doesn't improve for 50 rounds

    evals = [(dtrain, 'train'), (dval, 'eval')]
    progress = {}
    
    
    bst = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=evals,
        early_stopping_rounds=early_stopping_rounds,
        evals_result=progress
    )

    print("Best MAE: {:.2f} with {} rounds".format(
             bst.best_score,
             bst.best_iteration+1))
    
    predictions = bst.predict(test)


    return predictions
    # If you want to check the predictions
    # print(y_pred)


In [94]:
y_train_a = pd.read_csv('cleaned_data/A/train_a.csv')
y_train_b = pd.read_csv('cleaned_data/B/train_b.csv')
y_train_c = pd.read_csv('cleaned_data/C/train_c.csv')

In [95]:
xg_pred_a = XGBoost_model(X_train_a,y_train_a,X_test_a)
xg_pred_b = XGBoost_model(X_train_b,y_train_b,X_test_b)
xg_pred_c = XGBoost_model(X_train_c,y_train_c,X_test_c)

[0]	train-mae:826.67734	eval-mae:714.16229
[1]	train-mae:789.97960	eval-mae:682.75281
[2]	train-mae:755.43497	eval-mae:653.96886
[3]	train-mae:722.84769	eval-mae:626.65113
[4]	train-mae:692.16312	eval-mae:600.51901
[5]	train-mae:663.23749	eval-mae:576.71361
[6]	train-mae:636.12343	eval-mae:553.96715
[7]	train-mae:610.62857	eval-mae:532.82759
[8]	train-mae:586.67988	eval-mae:512.74372
[9]	train-mae:564.11596	eval-mae:493.64401
[10]	train-mae:542.76436	eval-mae:475.64179
[11]	train-mae:522.78424	eval-mae:458.75371
[12]	train-mae:503.94152	eval-mae:442.46680
[13]	train-mae:486.20842	eval-mae:427.06780
[14]	train-mae:469.45890	eval-mae:412.42452
[15]	train-mae:453.65408	eval-mae:398.31830
[16]	train-mae:438.66533	eval-mae:384.85708
[17]	train-mae:424.81664	eval-mae:372.33449
[18]	train-mae:411.62391	eval-mae:360.25343
[19]	train-mae:399.07077	eval-mae:348.89455
[20]	train-mae:387.19074	eval-mae:337.94788
[21]	train-mae:376.11187	eval-mae:327.23941
[22]	train-mae:365.70509	eval-mae:317.1745

[185]	train-mae:148.76826	eval-mae:133.43444
[186]	train-mae:148.61203	eval-mae:133.52607
[187]	train-mae:148.41727	eval-mae:133.63753
[188]	train-mae:148.24180	eval-mae:133.83557
[189]	train-mae:148.02849	eval-mae:133.83606
[190]	train-mae:147.76931	eval-mae:133.74982
[191]	train-mae:147.45831	eval-mae:133.77414
[192]	train-mae:147.16984	eval-mae:133.71060
[193]	train-mae:146.99302	eval-mae:133.40853
[194]	train-mae:146.72257	eval-mae:133.39892
[195]	train-mae:146.54669	eval-mae:133.42279
[196]	train-mae:146.33995	eval-mae:133.40673
[197]	train-mae:146.16046	eval-mae:133.26618
[198]	train-mae:145.87937	eval-mae:133.38658
[199]	train-mae:145.63723	eval-mae:133.67865
[200]	train-mae:145.45848	eval-mae:133.68088
[201]	train-mae:145.20789	eval-mae:133.48109
[202]	train-mae:145.07590	eval-mae:133.45060
[203]	train-mae:144.91147	eval-mae:133.45586
[204]	train-mae:144.70141	eval-mae:133.44970
[205]	train-mae:144.50077	eval-mae:133.42730
[206]	train-mae:144.25029	eval-mae:133.27922
[207]	trai

XGBoostError: [22:24:06] /Users/runner/work/xgboost/xgboost/src/data/data.cc:501: Check failed: this->labels.Size() % this->num_row_ == 0 (4102 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001604e8355 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x00000001605bec83 xgboost::MetaInfo::SetInfoFromHost(xgboost::Context const&, xgboost::StringView, xgboost::Json) + 3491
  [bt] (2) 3   libxgboost.dylib                    0x00000001605bddf2 xgboost::MetaInfo::SetInfo(xgboost::Context const&, xgboost::StringView, xgboost::StringView) + 146
  [bt] (3) 4   libxgboost.dylib                    0x0000000160500729 XGDMatrixSetInfoFromInterface + 233
  [bt] (4) 5   libffi.8.dylib                      0x000000010b9e9972 ffi_call_unix64 + 82
  [bt] (5) 6   ???                                 0x00000003066da930 0x0 + 12992751920



XGBoostError: [22:22:16] /Users/runner/work/xgboost/xgboost/src/data/data.cc:507: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001604e8355 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x00000001605bedb9 xgboost::MetaInfo::SetInfoFromHost(xgboost::Context const&, xgboost::StringView, xgboost::Json) + 3801
  [bt] (2) 3   libxgboost.dylib                    0x00000001605bddf2 xgboost::MetaInfo::SetInfo(xgboost::Context const&, xgboost::StringView, xgboost::StringView) + 146
  [bt] (3) 4   libxgboost.dylib                    0x0000000160500729 XGDMatrixSetInfoFromInterface + 233
  [bt] (4) 5   libffi.8.dylib                      0x000000010b9e9972 ffi_call_unix64 + 82
  [bt] (5) 6   ???                                 0x00000003066da930 0x0 + 12992751920



In [ ]:
xg

In [90]:
xgb_sub = create_sub(xg_pred_a,xg_pred,xg_pred_c)

NameError: name 'xg_pred_b' is not defined

In [62]:
def create_sub(preds_a,preds_b,preds_c):
    submission = pd.read_csv('sample_submission.csv')
    predictions = preds_a.rbind(preds_b).rbind(preds_c)
    predictions_df = predictions['predict'].as_data_frame()
    predictions_df.loc[predictions_df['predict'] < 0, 'predict'] = 0
    submission['prediction'] = predictions_df['predict']
    return submission 

In [59]:
submission.to_csv('Submissions/koman.csv', index=False)

In [52]:
predictions_df

,predict
0,22.157389
1,-1.091350
2,2.241392
3,6.769992
4,44.577548
...,...
2155,37.086241
2156,12.000001
2157,3.294035
2158,-1.778760


,id,prediction
0,0,22.157389
1,1,-1.091350
2,2,2.241392
3,3,6.769992
4,4,44.577548
...,...,...
2155,2155,37.086241
2156,2156,12.000001
2157,2157,3.294035
2158,2158,-1.778760


In [41]:
preds_b

predict
22.1574
-1.09135
2.24139
6.76999
44.5775
141.999
278.099
425.561
394.561
492.926


In [40]:
preds_a

predict
22.1574
-1.09135
2.24139
6.76999
44.5775
147.151
276.241
425.561
399.39
500.234
